In [1]:
%load_ext autoreload
%autoreload 2

import json
import sys
import os
base_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))
sys.path.append(base_path)

from evaluation.datasets import PopulatedSchemaDataset
import json
import glob
from pathlib import Path
from meri.meri import MERI
from jsonpath_ng.jsonpath import DatumInContext
from jsonpath_ng import parse
from typing import List
import fnmatch
import re

from full_pipeline_evaluator import build_jsonpath_queries, ExtractionResultItem, ExtractionResults

/root/miniconda3/envs/layout_analysis/lib/python3.12/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[0717 09:19.21 @file_utils.py:36]  INF  PyTorch version 2.3.0 available.


In [2]:
dataset = PopulatedSchemaDataset(json_schema_dir="/workspaces/meri/data/json_schema", gt_populated_dir="/workspaces/meri/data/gt_populated_schema", datasheets_dir="/workspaces/meri/data/datasheets")

In [3]:
pdf_path, schema, parameters_gt = dataset.__getitem__(0)
meri = MERI(pdf_path=pdf_path, config_yaml_path='/workspaces/meri/meri/configs/meri_default.yaml')

In [4]:
parameters_pred = meri.run(json_schema_string=json.dumps(schema))

[0717 09:19.24 @config.py:66]  WRN  Loading config /root/.cache/deepdoctection/configs/dd/d2/layout/CASCADE_RCNN_R_50_FPN_GN.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


Running configuration:  <bound method AttrDict.to_dict of {'DEVICE': 'cuda',
 'LIB': 'PT',
 'PT': {'LAYOUT': {'FILTER': ['table', 'figure'],
                   'WEIGHTS': 'layout/d2_model_0829999_layout_inf_only.pt'}}}>


[0717 09:19.26 @detection_checkpoint.py:38]  INF  [DetectionCheckpointer] Loading from /root/.cache/deepdoctection/weights/layout/d2_model_0829999_layout_inf_only.pt ...
[0717 09:19.26 @checkpoint.py:150]  INF  [Checkpointer] Loading from /root/.cache/deepdoctection/weights/layout/d2_model_0829999_layout_inf_only.pt ...
You are using a model of type table-transformer to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
[0717 09:19.26 @_builder.py:187]  INF  Loading pretrained weights from Hugging Face hub (timm/resnet18.a1_in1k)


Running configuration:  <bound method AttrDict.to_dict of {'DEVICE': 'cuda',
 'LIB': 'PT',
 'PT': {'LAYOUT': {'FILTER': None,
                   'WEIGHTS': 'microsoft/table-transformer-detection/pytorch_model.bin'}}}>


[0717 09:19.26 @_hub.py:180]  INF  [timm/resnet18.a1_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
[0717 09:19.26 @_builder.py:245]  INF  Missing keys (fc.weight, fc.bias) discovered while loading pretrained weights. This is expected if model is being adapted.
Some weights of the model checkpoint at /root/.cache/deepdoctection/weights/microsoft/table-transformer-detection/pytorch_model.bin were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a Ber

...

Generated markdown tables: ['<table title="" description=""></table>']
computing with character threshold:  1000000


Processing content chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Estimated input token:  2425


[0717 09:20.04 @_client.py:1026]  INF  HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing content chunks: 100%|██████████| 1/1 [00:07<00:00,  7.78s/it]

Actual Token Usage:  ('completion_tokens', 219) ('prompt_tokens', 5212) ('total_tokens', 5431)


In [5]:
parameters_pred

{'title': {'text': 'Danfoss HEXSelector 1.3.3',
  'bbox': [32.029998779296875, 97.654296875, 579.1057739257812, 106.642578125],
  'pageIndex': 0},
 'technicalSpecifications': {'HE16_PLATE_MATERIAL': {'value': 'AISI316L / 37%',
   'text': 'AISI316L / 37%',
   'bbox': [30, 420, 580, 695],
   'pageIndex': 0},
  'HE16_PLATE_THICKNESS': {'parameter_properties': {'value': '0.40',
    'unit': 'mm',
    'text': '2x 0,40 (Industry)',
    'bbox': [30, 420, 580, 695],
    'pageIndex': 0}}},
 'notFoundList': ['HE16_DESIGN_POWER',
  'HE16_HT_COEFF_1_2',
  'HE16_HT_COEFF_3_4',
  'HE16_PLATE_BREADTH',
  'HE16_PLATE_DISTANCE_1_2',
  'HE16_PLATE_DISTANCE_3_4',
  'HE16_PLATE_LENGTH',
  'HE16_PLATE_NUMBER']}

- check that prediction follows json schem
- create all json queries answered in gt and then check if they are in prediction

In [6]:
jsonpath_queries = build_jsonpath_queries(parameters_gt, ignore_keys=['notFoundList', 'bbox'], termination_keys = ['bbox'])


extacted_and_correct = ExtractionResults() # correctly extracted value
extacted_and_incorrect = ExtractionResults() # value extracted, but wrong
not_extracted = ExtractionResults() # value not extracted
total = 0

for query in jsonpath_queries:
    jsonpath_expr = parse(query)

    gt_res: List[DatumInContext] = jsonpath_expr.find(parameters_gt)
    pred_res: List[DatumInContext] = jsonpath_expr.find(parameters_pred)

    # queries are specific, no wildcards. So we only expect one result per query
    assert len(gt_res) == 1 and len(pred_res) <= 1

    ## check if value is dict, then skip. else evaluate
    gt_value = gt_res[0].value
    if isinstance(gt_value, dict):
        continue

    total += 1
    # check if query has response in prediction
    if len(pred_res) == 0:
        not_extracted.append(ExtractionResultItem(query, gt_value, None))
        continue

    pred_value = pred_res[0].value
    res_item = ExtractionResultItem(query, gt_value, pred_value)

    # TODO compute IOU

    if pred_value == gt_value:
        extacted_and_correct.append(res_item)
    else:
        extacted_and_incorrect.append(res_item)
    

In [7]:
x = not_extracted.children_as_group('$.technicalSpecifications')[1]
x.longest_subquery

'$.technicalSpecifications.HE16_HT_COEFF_1_2.parameter_properties'

In [8]:
extacted_and_correct.to_list()


[{'json_query': '$.title.text',
  'gt_value': 'Danfoss HEXSelector 1.3.3',
  'pred_value': 'Danfoss HEXSelector 1.3.3'},
 {'json_query': '$.title.pageIndex', 'gt_value': 0, 'pred_value': 0},
 {'json_query': '$.technicalSpecifications.HE16_PLATE_THICKNESS.parameter_properties.unit',
  'gt_value': 'mm',
  'pred_value': 'mm'},
 {'json_query': '$.technicalSpecifications.HE16_PLATE_THICKNESS.parameter_properties.pageIndex',
  'gt_value': 0,
  'pred_value': 0}]